# Challenge 2 - ANOVA

In statistics, **Analysis of Variance (ANOVA)** is also used to analyze the differences among group means. The difference between t-test and ANOVA is the former is ued to compare two groups whereas the latter is used to compare three or more groups. [Read more about the difference between t-test and ANOVA](http://b.link/anova24).

From the ANOVA test, you receive two numbers. The first number is called the **F-value** which indicates whether your null-hypothesis can be rejected. The critical F-value that rejects the null-hypothesis varies according to the number of total subjects and the number of subject groups in your experiment. In [this table](http://b.link/eda14) you can find the critical values of the F distribution. **If you are confused by the massive F-distribution table, don't worry. Skip F-value for now and study it at a later time. In this challenge you only need to look at the p-value.**

The p-value is another number yielded by ANOVA which already takes the number of total subjects and the number of experiment groups into consideration. **Typically if your p-value is less than 0.05, you can declare the null-hypothesis is rejected.**

In this challenge, we want to understand whether there are significant differences among various types of pokemons' `Total` value, i.e. Grass vs Poison vs Fire vs Dragon... There are many types of pokemons which makes it a perfect use case for ANOVA.

In [116]:
# Import libraries
import pandas as pd
import numpy as np
from scipy import stats

In [71]:
# Load the data:
data = pd.read_csv('Pokemon.csv')
data.head(41)

#                       Name    Type 1  Type 2  Total  HP  Attack  \
0    1                  Bulbasaur     Grass  Poison    318  45      49   
1    2                    Ivysaur     Grass  Poison    405  60      62   
2    3                   Venusaur     Grass  Poison    525  80      82   
3    3      VenusaurMega Venusaur     Grass  Poison    625  80     100   
4    4                 Charmander      Fire     NaN    309  39      52   
5    5                 Charmeleon      Fire     NaN    405  58      64   
6    6                  Charizard      Fire  Flying    534  78      84   
7    6  CharizardMega Charizard X      Fire  Dragon    634  78     130   
8    6  CharizardMega Charizard Y      Fire  Flying    634  78     104   
9    7                   Squirtle     Water     NaN    314  44      48   
10   8                  Wartortle     Water     NaN    405  59      63   
11   9                  Blastoise     Water     NaN    530  79      83   
12   9    BlastoiseMega Blastoise     Water     NaN    630  79     103   
13  10                   Caterpie       Bug     NaN    195  45      30   
14  11                    Metapod       Bug     NaN    205  50      20   
15  12                 Butterfree       Bug  Flying    395  60      45   
16  13                     Weedle       Bug  Poison    195  40      35   
17  14                     Kakuna       Bug  Poison    205  45      25   
18  15                   Beedrill       Bug  Poison    395  65      90   
19  15      BeedrillMega Beedrill       Bug  Poison    495  65     150   
20  16                     Pidgey    Normal  Flying    251  40      45   
21  17                  Pidgeotto    Normal  Flying    349  63      60   
22  18                    Pidgeot    Normal  Flying    479  83      80   
23  18        PidgeotMega Pidgeot    Normal  Flying    579  83      80   
24  19                    Rattata    Normal     NaN    253  30      56   
25  20                   Raticate    Normal     NaN    413  55      81   
26  21                    Spearow    Normal  Flying    262  40      60   
27  22                     Fearow    Normal  Flying    442  65      90   
28  23                      Ekans    Poison     NaN    288  35      60   
29  24                      Arbok    Poison     NaN    438  60      85   
30  25                    Pikachu  Electric     NaN    320  35      55   
31  26                     Raichu  Electric     NaN    485  60      90   
32  27                  Sandshrew    Ground     NaN    300  50      75   
33  28                  Sandslash    Ground     NaN    450  75     100   
34  29                   Nidoran♀    Poison     NaN    275  55      47   
35  30                   Nidorina    Poison     NaN    365  70      62   
36  31                  Nidoqueen    Poison  Ground    505  90      92   
37  32                   Nidoran♂    Poison     NaN    273  46      57   
38  33                   Nidorino    Poison     NaN    365  61      72   
39  34                   Nidoking    Poison  Ground    505  81     102   
40  35                   Clefairy     Fairy     NaN    323  70      45   

    Defense  Sp. Atk  Sp. Def  Speed  Generation  Legendary  
0        49       65       65     45           1      False  
1        63       80       80     60           1      False  
2        83      100      100     80           1      False  
3       123      122      120     80           1      False  
4        43       60       50     65           1      False  
5        58       80       65     80           1      False  
6        78      109       85    100           1      False  
7       111      130       85    100           1      False  
8        78      159      115    100           1      False  
9        65       50       64     43           1      False  
10       80       65       80     58           1      False  
11      100       85      105     78           1      False  
12      120      135      115     78           1      False  
13       35       20       2

**To achieve our goal, we use three steps:**

1. **Extract the unique values of the pokemon types.**

1. **Select dataframes for each unique pokemon type.**

1. **Conduct ANOVA analysis across the pokemon types.**

#### First let's obtain the unique values of the pokemon types. These values should be extracted from Type 1 and Type 2 aggregated. Assign the unique values to a variable called `unique_types`.

*Hint: the correct number of unique types is 19 including `NaN`. You can disregard `NaN` in next step.*

In [103]:
# Your code here
unique_types = list(set([v for val in data[['Type 1', 'Type 2']].values for v in val if type(v) == str]))
len(unique_types) # you should see 19
# excluded the nan

18

#### Second we will create a list named `pokemon_totals` to contain the `Total` values of each unique type of pokemons.

Why we use a list instead of a dictionary to store the pokemon `Total`? It's because ANOVA only tells us whether there is a significant difference of the group means but does not tell which group(s) are significantly different. Therefore, we don't need know which `Total` belongs to which pokemon type.

*Hints:*

* Loop through `unique_types` and append the selected type's `Total` to `pokemon_groups`.
* Skip the `NaN` value in `unique_types`. `NaN` is a `float` variable which you can find out by using `type()`. The valid pokemon type values are all of the `str` type.
* At the end, the length of your `pokemon_totals` should be 18.

In [115]:
# Your code here
pokemon_totals = [data[(data["Type 1"] == u) | (data["Type 2"] == u)]["Total"] for u in unique_types]
     
len(pokemon_totals) # you should see 18

18

#### Now we run ANOVA test on `pokemon_totals`.

*Hints:*

* To conduct ANOVA, you can use `scipy.stats.f_oneway()`. Here's the [reference](http://b.link/scipy44).

* What if `f_oneway` throws an error because it does not accept `pokemon_totals` as a list? The trick is to add a `*` in front of `pokemon_totals`, e.g. `stats.f_oneway(*pokemon_groups)`. This trick breaks the list and supplies each list item as a parameter for `f_oneway`.

In [118]:
# Your code here
stats.f_oneway(*pokemon_totals)

F_onewayResult(statistic=6.6175382960055344, pvalue=2.6457458815984803e-15)

#### Interpret the ANOVA test result. Is the difference significant?

In [ ]:
# Your comment here
# The pvalue is way below 5%, so it has relevant difference.